In [1]:
%load_ext autoreload
%autoreload 2

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch.utils.data import DataLoader
# import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
import random
from itertools import cycle
from dotenv import load_dotenv
import os
import tqdm.notebook as tqdm
# from peft import PeftModel, PeftConfig, get_peft_model
# from datasets import load_dataset
# import wandb
from tqdm import tqdm
import json

load_dotenv()
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")




from dataclasses import asdict
from datetime import datetime


exp_time = datetime.now().strftime("%b%d")


/root/.venv/multi2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=TORCH_TYPE,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

device_map = infer_auto_device_map(
    model=model,
    max_memory={i: "23GiB" for i in range(torch.cuda.device_count())},
    # set 23GiB for each GPU, depends on your GPU memory, you can adjust this value
    no_split_module_classes=["CogVLMDecoderLayer"]
)
# checkpoint = torch.load(MODEL_PATH, map_location='cpu')
# save_dir = "/root/multimodal-eng/models"
# model.save_pretrained(save_dir)
checkpoint = "/root/multimodal-eng/models/24may-1"
# model = load_checkpoint_and_dispatch(model, MODEL_PATH, device_map=device_map, dtype=TORCH_TYPE)
model = load_checkpoint_and_dispatch(model, checkpoint, device_map=device_map, dtype=TORCH_TYPE)
model = model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 8/8 [00:01<00:00,  7.76it/s]


: 

In [4]:
from transformers.utils import cached_path, hf_bucket_url

# Get the URL to the model's configuration file
MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
config_url = hf_bucket_url(MODEL_PATH, filename="config.json")

# Get the local cache path for the configuration file
local_config_path = cached_path(config_url)

# Extract the directory path
model_dir = os.path.dirname(local_config_path)
print(f"Model directory: {model_dir}")

ImportError: cannot import name 'cached_path' from 'transformers.utils' (/root/.venv/multi2/lib/python3.10/site-packages/transformers/utils/__init__.py)

In [10]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=TORCH_TYPE,
    trust_remote_code=True,
).to(DEVICE).eval()

text_only_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {} ASSISTANT:"

while True:
    image_path = input("image path >>>>> ")
    if image_path == '':
        print('You did not enter image path, the following will be a plain text conversation.')
        image = None
        text_only_first_query = True
    else:
        image = Image.open(image_path).convert('RGB')

    history = []

    while True:
        query = input("Human:")
        if query == "clear":
            break

        if image is None:
            if text_only_first_query:
                query = text_only_template.format(query)
                text_only_first_query = False
            else:
                old_prompt = ''
                for _, (old_query, response) in enumerate(history):
                    old_prompt += old_query + " " + response + "\n"
                query = old_prompt + "USER: {} ASSISTANT:".format(query)
        if image is None:
            input_by_model = model.build_conversation_input_ids(
                tokenizer,
                query=query,
                history=history,
                template_version='chat'
            )
        else:
            input_by_model = model.build_conversation_input_ids(
                tokenizer,
                query=query,
                history=history,
                images=[image],
                template_version='chat'
            )
        inputs = {
            'input_ids': input_by_model['input_ids'].unsqueeze(0).to(DEVICE),
            'token_type_ids': input_by_model['token_type_ids'].unsqueeze(0).to(DEVICE),
            'attention_mask': input_by_model['attention_mask'].unsqueeze(0).to(DEVICE),
            'images': [[input_by_model['images'][0].to(DEVICE).to(TORCH_TYPE)]] if image is not None else None,
        }
        gen_kwargs = {
            "max_new_tokens": 2048,
            "pad_token_id": 128002,  
        }
        with torch.no_grad():
            outputs = model.generate(**inputs, **gen_kwargs)
            outputs = outputs[:, inputs['input_ids'].shape[1]:]
            response = tokenizer.decode(outputs[0])
            response = response.split("<|end_of_text|>")[0]
            print("\nCogVLM2:", response)
        history.append((query, response))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/THUDM/cogvlm2-llama3-chat-19B:
- configuration_cogvlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/cogvlm2-llama3-chat-19B:
- visual.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/cogvlm2-llama3-chat-19B:
- util.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded


CogVLM2: You are looking at a picturesque scene of a wooden boardwalk winding through a lush expanse of tall green grass, leading the viewer's eye towards a distant horizon dotted with trees. The sky above is a vivid blue with wispy clouds scattered about, suggesting a clear and sunny day. The overall atmosphere of the image is one of tranquility and natural beauty, evoking feelings of serenity and a deep connection with the environment.

CogVLM2: The image doesn't provide explicit geographical markers or distinctive landmarks that would pinpoint an exact location. However, the landscape is reminiscent of wetland or marshy areas, which are common in many parts of the world. Such environments can be found in North America, Europe, Asia, and Australia, among others. The vegetation suggests a temperate climate, but without more context, it's challenging to determine the exact country or region where this photo was taken.

CogVLM2: Determining the exact location where this picture was tak

KeyboardInterrupt: 

: 

# getting batched inference

In [1]:
"""
This is a demo for using CogVLM2 in CLI using multi-GPU with lower memory.
If your single GPU is not enough to drive this model, you can use this demo to run this model on multiple graphics cards with limited video memory.
Here, we default that your graphics card has 24GB of video memory, which is not enough to load the FP16 / BF16 model.
so , need to use two graphics cards to load. We set '23GiB' for each GPU to avoid out of memory.
"""
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map
import os
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import textwrap

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[
    0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=TORCH_TYPE,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

device_map = infer_auto_device_map(
    model=model,
    max_memory={i: "23GiB" for i in range(torch.cuda.device_count())},
    # set 23GiB for each GPU, depends on your GPU memory, you can adjust this value
    no_split_module_classes=["CogVLMDecoderLayer"]
)
checkpoint = "/root/multimodal-eng/models/24may-1"
model = load_checkpoint_and_dispatch(model, checkpoint, device_map=device_map, dtype=TORCH_TYPE)
model = model.eval()
print("Model loaded.")

text_only_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {} ASSISTANT:"

image_folder = "/root/multimodal-eng/pics_of_mates_faces/"
user_input = "Describe this person to me."

images_and_responses = []


/root/.venv/multi2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 8/8 [00:00<00:00, 10.90it/s]


Model loaded.


In [2]:
batched_inputs = []

for idx, image_file in tqdm(enumerate(os.listdir(image_folder))):
    if not image_file.endswith(".png"):
        continue
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path).convert('RGB')

    history = []

    input_by_model = model.build_conversation_input_ids(
        tokenizer,
        query=user_input,
        images=[image],
        template_version='chat'
    )
    inputs = {
        'input_ids': input_by_model['input_ids'].unsqueeze(0).to(DEVICE),
        'token_type_ids': input_by_model['token_type_ids'].unsqueeze(0).to(DEVICE),
        'attention_mask': input_by_model['attention_mask'].unsqueeze(0).to(DEVICE),
        'images': [[input_by_model['images'][0].to(DEVICE).to(TORCH_TYPE)]] if image is not None else None,
    }

    batched_inputs.append(inputs)

batched_input_ids = torch.cat([inputs['input_ids'] for inputs in batched_inputs], dim=0)


17it [00:02,  7.42it/s]


In [4]:
batched_input_ids.shape

torch.Size([16, 2317])

In [9]:
with torch.no_grad():
    outputs = model.generate(
        input_ids=batched_input_ids[:4, :],
        token_type_ids=torch.cat([inputs['token_type_ids'] for inputs in batched_inputs[:4]], dim=0),
        max_new_tokens=2048,
        pad_token_id=128002,
        images=[inputs['images'][0] for inputs in batched_inputs[:4]]
    )
for output_idx, outputs in enumerate(outputs):
    response = tokenizer.decode(outputs)
    response = response.split("<|end_of_text|>")[0].split("Question:")[-1]
    print(response)

 Describe this person to me. Answer:The image depicts a person with a pronounced facial expression, characterized by an open mouth as if they are shouting or expressing surprise. The skin tone appears to be fair, and the individual has short, light-colored hair. The eyes are partially visible, but due to the angle and the open mouth, it's difficult to discern the full expression of the eyes. The background is blurred, making it hard to determine the context or setting of the image.
 Describe this person to me. Answer:The person in the image is wearing a pair of sunglasses with a tinted lens, and they appear to have a fair complexion. The individual's hair is short, with a slightly messy or tousled appearance. Due to the blurred nature of the image, it is challenging to discern more specific details or facial features.
 Describe this person to me. Answer:This image depicts a young individual with a youthful appearance. They have light skin and curly brown hair that is slightly tousled, 